In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
#names_df = pd.read_csv("Resources/yob1978.txt", delimiter =",", names=["Name", "Gender", "YearOfBirth"],dtype={"Name":"str","Gender":"str", "YearOfBirth":np.int32})

start_year=1881
end_year=2019
yearlist= list(range(start_year,end_year,1))
#print(yearlist)
year_df=pd.DataFrame(yearlist)
year_df["dummy_col"]=1
year_df.set_index("dummy_col", inplace=True)
filename_list=[]
for file in os.listdir("Resources"):
    if file.endswith(".txt") and int(file[3:7]) >= start_year:
        df =  pd.read_csv("Resources/"+file, delimiter =",", names=["Name", "Gender", "CountOfNames"],dtype={"Name":"str","Gender":"str", "CountOfNames":np.int32})
        df["Year"] = int(file[3:7])
        
        df = df[["Year", "Name", "Gender", "CountOfNames"]]
        filename_list.append(df)
        
names_df = pd.concat(filename_list, axis=0, ignore_index=True)
names_df.sort_values(by=["Name","Gender","Year"], inplace=True)
unique_names_list = names_df.drop_duplicates(subset=["Name", "Gender"])[["Name", "Gender"]]
unique_names_list["dummy_col"]=1
unique_names_list.set_index("dummy_col", inplace=True)

final_data_set = unique_names_list.join(year_df,on="dummy_col",how = "outer")
final_data_set.count()
final_data_set.tail(10)

,Name,Gender,0
dummy_col,,,
1,Zzyzx,M,2009
1,Zzyzx,M,2010
1,Zzyzx,M,2011
1,Zzyzx,M,2012
1,Zzyzx,M,2013
1,Zzyzx,M,2014
1,Zzyzx,M,2015
1,Zzyzx,M,2016
1,Zzyzx,M,2017


In [4]:
df = final_data_set.rename(columns={0:"Year"})
df.groupby(["Name","Gender"]).count()

,,Year
Name,Gender,
Aaban,M,138
Aabha,F,138
Aabid,M,138
Aabidah,F,138
Aabir,M,138
Aabriella,F,138
Aada,F,138
Aadam,M,138
Aadan,M,138


In [6]:
df.loc[df["Name"]=="Aaban"]

,Name,Gender,Year
dummy_col,,,
1,Aaban,M,1881
1,Aaban,M,1882
1,Aaban,M,1883
1,Aaban,M,1884
1,Aaban,M,1885
1,Aaban,M,1886
1,Aaban,M,1887
1,Aaban,M,1888
1,Aaban,M,1889


In [ ]:
unique_names_list

In [ ]:
names_df.head()